In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import random
import time
import warnings

from tqdm import tqdm

# Suppress unnecessary FutureWarning
warnings.filterwarnings('ignore', category=FutureWarning)

In [2]:
topics = pd.read_csv('/kaggle/input/nir-2024/sampled_train_topics.csv')
qrels = pd.read_csv('/kaggle/input/nir-2024/sampled_train_qrels.csv')

In [3]:
!curl -s "https://get.sdkman.io" | bash && source "$HOME/.sdkman/bin/sdkman-init.sh" && sdk install java 11.0.22-amzn < /dev/null


                                -+syyyyyyys:
                            `/yho:`       -yd.
                         `/yh/`             +m.
                       .oho.                 hy                          .`
                     .sh/`                   :N`                `-/o`  `+dyyo:.
                   .yh:`                     `M-          `-/osysoym  :hs` `-+sys:      hhyssssssssy+
                 .sh:`                       `N:          ms/-``  yy.yh-      -hy.    `.N-````````+N.
               `od/`                         `N-       -/oM-      ddd+`     `sd:     hNNm        -N:
              :do`                           .M.       dMMM-     `ms.      /d+`     `NMMs       `do
            .yy-                             :N`    ```mMMM.      -      -hy.       /MMM:       yh
          `+d+`           `:/oo/`       `-/osyh/ossssssdNMM`           .sh:         yMMN`      /m.
         -dh-           :ymNMMMMy  `-/shmNm-`:N/-.``   `.sN            /N-         `NMMy      .m/
  

In [4]:
pip install python-terrier

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.9/337.9 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 40.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.8/1

In [5]:
%env JAVA_HOME=/root/.sdkman/candidates/java/current
import pyterrier as pt
if not pt.started():
    pt.init()

env: JAVA_HOME=/root/.sdkman/candidates/java/current
terrier-assemblies 5.9 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.8 jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.10.1 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [6]:
dataset = pt.get_dataset('irds:dpr-w100')

In [7]:
# Remove stopwords
start_time = time.time()

indexer =  pt.IterDictIndexer("./indexes/stopwords")
indexer.setProperty("termpipelines", "Stopwords")
index_ref = indexer.index(dataset.get_corpus_iter(), fields=['text', 'title'])
index_ref = pt.IndexRef.of("./indexes/stopwords/data.properties")
index = pt.IndexFactory.of(index_ref)
print(index.getCollectionStatistics().toString())

end_time = time.time()
print(end_time - start_time)
len((index.getLexicon()))

[INFO] If you have a local copy of https://dl.fbaipublicfiles.com/dpr/wikipedia_split/psgs_w100.tsv.gz, you can symlink it here to avoid downloading it again: /root/.ir_datasets/downloads/612fe66e0b6b41ee28f806140226c563
[INFO] [starting] https://dl.fbaipublicfiles.com/dpr/wikipedia_split/psgs_w100.tsv.gz
dpr-w100 documents:   0%|          | 0/21015324 [00:00<?, ?it/s]
https://dl.fbaipublicfiles.com/dpr/wikipedia_split/psgs_w100.tsv.gz: 0.0%| 0.00/4.69G [00:00<?, ?B/s]
https://dl.fbaipublicfiles.com/dpr/wikipedia_split/psgs_w100.tsv.gz: 0.1%| 4.92M/4.69G [00:00<01:36, 48.4MB/s]
https://dl.fbaipublicfiles.com/dpr/wikipedia_split/psgs_w100.tsv.gz: 0.3%| 13.4M/4.69G [00:00<01:10, 66.6MB/s]
https://dl.fbaipublicfiles.com/dpr/wikipedia_split/psgs_w100.tsv.gz: 0.6%| 26.2M/4.69G [00:00<00:53, 86.8MB/s]
https://dl.fbaipublicfiles.com/dpr/wikipedia_split/psgs_w100.tsv.gz: 0.8%| 39.0M/4.69G [00:00<00:47, 97.0MB/s]
https://dl.fbaipublicfiles.com/dpr/wikipedia_split/psgs_w100.tsv.gz: 1.1%| 51.8M/4

20:23:43.501 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading lookup file directly from disk (SLOW) - try index.meta.index-source=fileinmem in the index properties file. 137.3 MiB of memory would be required.
20:23:43.560 [ForkJoinPool-1-worker-3] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 401.4 MiB of memory would be required.
Number of documents: 21015324
Number of terms: 4630820
Number of postings: 1030301407
Number of fields: 2
Number of tokens: 1286954378
Field names: [text, title]
Positions:   false

7578.701822042465


4630820

In [8]:
index_ref = pt.IndexRef.of("./indexes/stopwords/data.properties")
index = pt.IndexFactory.of(index_ref)
print(index.getCollectionStatistics().toString())

Number of documents: 21015324
Number of terms: 4630820
Number of postings: 1030301407
Number of fields: 2
Number of tokens: 1286954378
Field names: [text, title]
Positions:   false



In [9]:
search_times = []

# randomly selected 100 items
ix_range = random.sample(range(len(index.getLexicon())), 100)

# timimg
for ix, kv in enumerate(index.getLexicon()):
    start_time = time.time()
    if ix in ix_range:
        pointer = index.getLexicon()[kv.getKey()]
    end_time = time.time()
    search_time = end_time - start_time
    search_times.append(search_time)
    
print(sum(search_times)/100)

0.07854745626449584


In [11]:
topics["qid"] = topics["qid"].astype(str)
qrels["qid"] = qrels["qid"].astype(str)
qrels["docno"] = qrels["docno"].astype(str)

In [12]:
from sklearn.model_selection import KFold

# 5-folds
k = 5
topics_shuffled = topics.sample(frac=1, random_state=34)

In [13]:
type(topics[0:6000])

pandas.core.frame.DataFrame

In [14]:
topics_list = []

for i in range(k):
    # Calculate start and end indices for the current subset
    start_idx = i * 6000
    end_idx = start_idx + 6000
    
    # Extract the subset from the shuffled dataset
    subset = topics[start_idx:end_idx]
    
    topics_list.append(subset)

In [15]:
print(topics_list[1])

       Unnamed: 0     qid                                              query
6000        14274     nan  when was the last time essendon won a grand final
6001        61192     nan               what movie is the song all star from
6002        10315     nan    who took part in removing the stolen generation
6003        54704     nan     when was the first touch screen phone released
6004        35028     nan           when did mike trout come into the league
...           ...     ...                                                ...
11995       30514     nan        when did michigan become a state in the usa
11996       19497     nan  how many episodes in season 3 of fear of the w...
11997       35612     nan         piper orange is the new black actress name
11998       54127     nan  what important event took place each year that...
11999        6802  6123.0    when does my kitchen rules australia 2018 start

[6000 rows x 3 columns]


In [16]:
BM25_stopwords = pt.BatchRetrieve(index, wmodel="BM25",
                             controls={"bm25.b" : 0.75, "bm25.k_1": 1.2})

In [ ]:
from pyterrier.measures import *

pt.Experiment(
    retr_systems=[BM25_stopwords],
    names = ["BM25"],
    topics=topics_list[0],
    qrels=qrels,
    eval_metrics=[NDCG@10]
)

20:42:40.092 [main] WARN org.terrier.structures.FSADocumentIndex - This index has fields, but FSADocumentIndex is used (which stores fields lengths on disk); If using field-based models such as BM25F, change to index.document.class in the index  properties file to FSAFieldDocumentIndex or FSADocumentIndexInMemFields to support efficient retrieval. If you don't use (e.g.) BM25F, this warning can be ignored
20:42:44.745 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading lookup file directly from disk (SLOW) - try index.meta.index-source=fileinmem in the index properties file. 160.3 MiB of memory would be required.
20:42:44.750 [main] WARN org.terrier.structures.BaseCompressingMetaIndex - Structure meta reading data file directly from disk (SLOW) - try index.meta.data-source=fileinmem in the index properties file. 470.4 MiB of memory would be required.


In [18]:
from pyterrier.measures import *

tuned_BM25_stem, _ = pt.KFoldGridSearch(
    BM25_stem,
    params = {BM25_stem: {"bm25.b"  : [0.45, 0.55, 0.65, 0.85, 0.95, 1.05],
            "bm25.k_1": [0.6, 0.8, 1.0, 1.4, 1.6, 1.8]
    }},
    batch_size = 50,
    topics_list = topics_list,
    qrels = qrels,
    metric = NDCG@10
)
#pt.Experiment([pl2, tuned_pl2], all_topics, qrels, ["map"])

Fold 1


KeyboardInterrupt: 

In [ ]:
from pyterrier.measures import *
pt.Experiment(
    retr_systems=[BM25],
    names = ["BM25"],
    topics=topics,
    qrels=qrels,
    eval_metrics=[NDCG@10, 'map']
)